## Скрипт редактирования точек

- При нажатии на кнопку мыши выделяется точка с номером дерева, точек можно выделить сколько угодно.
- После нажатия кнопки Сложение точек, облака точек в датафрейме становятся одним деревом с наименьшим номером дерева из выделенных, после сложения точек отменить действие нельзя.
- Если неправильно выбраны точки, то можно их сбросить кнопкой сброс, но после объединения отменить действие нельзя.
- При нажатии на кнопку сохранения датасет сохраняется в csv файл
- Можно выделить дерево или несколько деревьев и сохранить каждое в отдельный файл с именем ( номером ) дерева.


для работы нужен ipywidgets 7.xx ( на 8 выдает ошибки)


In [5]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import ipywidgets as widgets

# загрузка сохраненного участка
plg = 'JS059_12'
plg_filename = f'D:/RTL/{plg}.csv'
polygon_data = pd.read_csv(plg_filename, sep=';')

# фильтрация данных по условию treeID > 0
polygon_data = polygon_data[polygon_data['treeID'] > 0]

unique_tree_ids = polygon_data['treeID'][~np.isnan(polygon_data['treeID']
                                                   )].unique()
print('Всего деревьев:', len(unique_tree_ids))

# создание Scatter объекта из DataFrame
f = go.FigureWidget(
    [go.Scatter(x=polygon_data['X'], y=polygon_data['Y'], mode='markers')])

scatter = f.data[0]

# создание словаря цветов для treeID
tree_id_colors = {}
unique_tree_ids = polygon_data['treeID'].unique()
for i in range(len(unique_tree_ids)):
    color = np.random.rand(3)
    tree_id_colors[unique_tree_ids[i]] = color

# применение цветов к точкам
colors = [
    'rgb(' + str(int(c[0] * 255)) + ',' + str(int(c[1] * 255)) + ',' +
    str(int(c[2] * 255)) +
    ')' if not np.isnan(np.array(c)).any() else 'rgb(255, 255, 255)' for c in [
        tree_id_colors.get(tree_id, [np.nan, np.nan, np.nan])
        for tree_id in polygon_data['treeID']
    ]
]

scatter.marker.color = colors

# добавление информации о treeID и Типе дерева при наведении на точку
scatter.text = polygon_data.apply(lambda row: f"treeID: {row['treeID']}, Тип дерева: {row['Tree_type']}", axis=1)

# создаем вспомогательные датафреймы
points_df = pd.DataFrame({'x': [], 'y': []})
treeID_df = pd.DataFrame({'treeID': []})
out_df = pd.DataFrame({'treeID': []})


# callback для добавления treeID по клику мыши
def update_point(trace, points, selector):
    # получение координат курсора мыши
    coordinates = [(trace.x[i], trace.y[i]) for i in points.point_inds]

    # добавление координат DataFrame
    new_rows = pd.DataFrame(coordinates, columns=['x', 'y'])
    global points_df
    points_df = pd.concat([points_df, new_rows])
    global treeID_df
    treeID_df = points_df.merge(polygon_data,
                                how='left',
                                left_on=['x', 'y'],
                                right_on=['X', 'Y'])[['treeID']]

    # получаем treeID и выводим сообщение

    treeID = treeID_df.iloc[-1]['treeID']
    print(f' Выделено дерево № {int(treeID)} ')

polygon_data2 = polygon_data.copy()  

# Создание кнопки и добавление функционала для объединения деревьев
def combine_points(b):
    global out_df
    # получаем minimum treeID из treeID_df
    min_treeID = treeID_df['treeID'].min()

    # проверяем, есть ли строки в treeID_df
    if treeID_df.empty:
        print("Нет данных для объединения.")
        return

    # получаем minimum treeID из treeID_df
    min_treeID = treeID_df['treeID'].min()
    if pd.isna(min_treeID):  # если min_treeID = NaN
        print("Нет данных для объединения.")
        return

    #в датафрейме polygon_data2 все значения treeID из treeID_df заменить на min_treeID
    polygon_data2.loc[polygon_data2['treeID'].isin(treeID_df['treeID']),
                      'treeID'] = min_treeID

    # выводим сообщение
    print(f'Точки объединены и им присвоен номер {int(min_treeID)}')

    # удаляем все точки из вспомогательных датафреймов
    points_df.drop(index=points_df.index, inplace=True)
    treeID_df.drop(index=treeID_df.index, inplace=True)


# Параметры красивостей для кнопки
combine_button = widgets.Button(description='Сложение точек')
combine_button.style.button_color = 'lightblue'  # Set button color
combine_button.style.font_weight = 'bold'  # Set font weight
combine_button.style.border = '2px solid gray'  # Set border
combine_button.style.border_radius = '5px'  # Set border radius
combine_button.on_click(combine_points)


# Создание кнопки и добавление функционала для сохранения в файл
def save_to_csv(b):
    global polygon_data2
    polygon_data2.to_csv(plg_filename,
                         sep=';',
                         index=False,
                         encoding='utf-8-sig')
    print('Файл успешно сохранен')

# Параметры красивостей для кнопки сохранения
save_button = widgets.Button(description='Сохр. файла')
save_button.style.button_color = 'lightgreen'  # Set button color
save_button.style.font_weight = 'bold'  # Set font weight
save_button.style.border = '2px solid gray' # Set border
save_button.style.border_radius = '5px' # Set border radius
save_button.on_click(save_to_csv)

# Создание кнопки и добавление функционала для сохранения одного дерева в один файл
def tree_to_csv(b):
    global polygon_data2
    # Получаем выбранные значения treeID из treeID_df
    selected_treeIDs = treeID_df['treeID']

    # проверяем, есть ли строки в treeID_df
    if treeID_df.empty:
        print('Нет данных для обработки.')
        return

    # сохраняем данные из polygon_data2 в новых файлах с именами treeID
    for treeID in selected_treeIDs:
        tree_data = polygon_data2[polygon_data2['treeID'] == treeID]
        if tree_data.empty:
            print(f'Данные для дерева с treeID {treeID} не найдены.')
            continue
        filename = f"D:/RTL/Trees/{int(treeID)}.csv"
        tree_data.to_csv(filename, sep=';', index=False, encoding='utf-8-sig')
        print(f'Данные для дерева с treeID = {int(treeID)} успешно сохранены в файле {filename}.')
        
# Параметры красивостей для кнопки сохранения
tree_button = widgets.Button(description='Сохр. дерева')
tree_button.style.button_color = 'lightyellow'  # Set button color
tree_button.style.font_weight = 'bold'  # Set font weight
tree_button.style.border = '2px solid gray' # Set border
tree_button.style.border_radius = '5px' # Set border radius
tree_button.on_click(tree_to_csv)

# Функция для сброса значений в points_df и treeID_df
def reset_points(b):
    global points_df, treeID_df
    points_df = pd.DataFrame(columns=['x', 'y', 'color'])
    treeID_df = pd.DataFrame(columns=['treeID'])
    print('Данные сброшены')

# Параметры красивостей для кнопки "Сброс"
reset_button = widgets.Button(description='Сброс')
reset_button.style.button_color = 'lightcoral'  # Set button color
reset_button.style.font_weight = 'bold'  # Set font weight
reset_button.style.border = '2px solid gray'  # Set border
reset_button.style.border_radius = '5px'  # Set border radius
reset_button.on_click(reset_points)

# Создаем виджет Dropdown
tree_dropdown = widgets.Dropdown(
    options=['Не определено', 'Ель', 'Сосна', 'Береза', 'Осина',
             'Ольха', 'Лиственница','Др. хвойные', 'Ива'],
    value='Не определено',
    #description='Дерево: ',
)

# Создаем кнопку
add_tree_button = widgets.Button(
    description='Тип дерева',
    button_style='success',  # меняем цвет кнопки на зеленый
)


# Обработчик нажатия на кнопку
def add_tree_column(button):
    # Получаем выбранное значение из Dropdown
    tree_type = tree_dropdown.value

    # Получаем выбранные значения treeID из treeID_df
    selected_treeIDs = treeID_df['treeID']

    # проверяем, есть ли строки в treeID_df
    if treeID_df.empty:
        print('Нет данных для присвоения типа.')
        return

    # Присваиваем выбранное значение типа дерева выбранным treeID в polygon_data2
    polygon_data2.loc[polygon_data2['treeID'].isin(selected_treeIDs),
                      'Tree_type'] = tree_type

    # Выводим сообщение
    print(f'Выбранным деревьям присвоено значение {tree_type}')


# Привязываем обработчик нажатия на кнопку
add_tree_button.on_click(add_tree_column)

# Объединяем кнопки в один контейнер
buttons_container = widgets.HBox([
    combine_button,
    tree_dropdown,
    add_tree_button,
    reset_button,
    save_button,
    tree_button,
])
display(buttons_container)

# добавляем callback function to scatter plot
scatter.on_click(update_point)

f.update_layout(xaxis_showticklabels=False, yaxis_showticklabels=False)

# задаем масштаб x к y
f.update_layout(
    width=600, # для регулировки окна вывода
    height=600,
    xaxis=dict(scaleanchor='y', scaleratio=1),
    yaxis=dict(scaleanchor='x', scaleratio=1),
)
# выводим на экран
f

Всего деревьев: 48


FigureWidget({
    'data': [{'marker': {'color': [rgb(154,158,145), rgb(154,158,145),
                        …